In [1]:
import openai
from dotenv import load_dotenv, find_dotenv


_ = load_dotenv(find_dotenv())

client = openai.Client()

In [2]:
import openai
print(openai.__version__)

1.95.1


In [3]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

assistant = client.beta.assistants.create(
    name="Tutor de Matemática",
    instructions="Você é um tutor pessoal de matemática. Escreva e execute códigos para responder perguntas matemáticas.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview"
)


Criação de um Thread

In [4]:
thread = client.beta.threads.create()

C:\Users\chris\AppData\Local\Temp\ipykernel_10844\3600338518.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()


Adicionar mensagem à thread criada

In [7]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content='se eu jogar um dado de 8 lados 1000 vezes, qual a probabilidade de cair o número 5 400 vezes? Resolva com um código'
)

C:\Users\chris\AppData\Local\Temp\ipykernel_10844\621382973.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\chris\AppData\Local\Temp\ipykernel_10844\621382973.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


Rodar a Thread no assistant

In [10]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions='O nome do usuário é Cole Palmer'
)

C:\Users\chris\AppData\Local\Temp\ipykernel_10844\594270314.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


Aguardar a Thread rodar

In [15]:

import time


while run.status in ['queued','in_progress', 'cancelling']:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )


C:\Users\chris\AppData\Local\Temp\ipykernel_10844\641565305.py:6: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


In [16]:
run.status

'completed'

Verificar a Resposta

In [17]:
if run.status =='completed':
    messages= client.beta.threads.messages.list(
        thread_id=thread.id
    )

    print(messages)

else:
    print('Erro', run.status)

C:\Users\chris\AppData\Local\Temp\ipykernel_10844\1184797989.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages= client.beta.threads.messages.list(


SyncCursorPage[Message](data=[Message(id='msg_Gxmf7QwC2nMzcTauIgHrs7KC', assistant_id='asst_xCb92lBBkWRhiPhGbDGpdXq3', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='A probabilidade de obter o número 5 exatamente 400 vezes ao lançar um dado de 8 lados 1000 vezes é incrivelmente baixa, aproximadamente \\(4.62 \\times 10^{-106}\\). Isso é um número muito próximo de zero, o que indica que é um evento extremamente improvável.'), type='text')], created_at=1752452260, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_R124P4G3djmpGQCKQgh2dAEm', status=None, thread_id='thread_Q6pjwphyIXlaZx4mAy5jaFIU'), Message(id='msg_MTJPe9yI3OaA1KfIC4ceyXvF', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='se eu jogar um dado de 8 lados 1000 vezes, qual a probabilidade de cair o número 5 400 vezes? Resolva com um código'), type='text')

In [18]:
print(messages.data[0].content[0].text.value)

A probabilidade de obter o número 5 exatamente 400 vezes ao lançar um dado de 8 lados 1000 vezes é incrivelmente baixa, aproximadamente \(4.62 \times 10^{-106}\). Isso é um número muito próximo de zero, o que indica que é um evento extremamente improvável.


### Analisando os passos do modelo

In [19]:
run_steps= client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
    
)




C:\Users\chris\AppData\Local\Temp\ipykernel_10844\687884520.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_steps= client.beta.threads.runs.steps.list(


In [21]:
run_steps.data[0]

RunStep(id='step_l9bX1iwXSPQGQ5d2Qc1kpJIl', assistant_id='asst_xCb92lBBkWRhiPhGbDGpdXq3', cancelled_at=None, completed_at=1752452262, created_at=1752452260, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_R124P4G3djmpGQCKQgh2dAEm', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_Gxmf7QwC2nMzcTauIgHrs7KC'), type='message_creation'), thread_id='thread_Q6pjwphyIXlaZx4mAy5jaFIU', type='message_creation', usage=Usage(completion_tokens=75, prompt_tokens=290, total_tokens=365, prompt_token_details={'cached_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0}), expires_at=None)

In [24]:
for step in run_steps.data:
    print("=== Step:", step.step_details.type)
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            print("-----------")
            print(tool_call.code_interpreter.input)
            print("-----------")
            print("Result")
            # Verifica se há outputs antes de acessar
            if tool_call.code_interpreter.outputs and len(tool_call.code_interpreter.outputs) > 0:
                print(tool_call.code_interpreter.outputs[0].logs)
            else:
                print("Nenhum log de saída disponível para este tool_call.")

=== Step: message_creation
=== Step: tool_calls
-----------
from scipy.stats import binom

# Definir os parâmetros do problema
n = 1000  # número de jogadas
k = 400   # número de sucessos desejado (obter um 5)
p = 1/8   # probabilidade de obter um 5 em uma jogada de um dado de 8 lados

# Calcula a probabilidade usando a função de massa de probabilidade da binomial
probabilidade = binom.pmf(k, n, p)

probabilidade
-----------
Result
Nenhum log de saída disponível para este tool_call.


In [27]:
for step in run_steps.data[::-1]:
    print("\n=== Step:", step.step_details.type)
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            print("-----------")
            print(tool_call.code_interpreter.input)
            print("-----------")
            print("Result")
            # Verifica se há outputs antes de acessar
            if tool_call.code_interpreter.outputs and len(tool_call.code_interpreter.outputs) > 0:
                print(tool_call.code_interpreter.outputs[0].logs)
            else:
                print("Nenhum log de saída disponível para este tool_call.")

    if step.step_details.type == 'message_creation':
       message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id)
    print(message.content[0].text.value)


=== Step: tool_calls
-----------
from scipy.stats import binom

# Definir os parâmetros do problema
n = 1000  # número de jogadas
k = 400   # número de sucessos desejado (obter um 5)
p = 1/8   # probabilidade de obter um 5 em uma jogada de um dado de 8 lados

# Calcula a probabilidade usando a função de massa de probabilidade da binomial
probabilidade = binom.pmf(k, n, p)

probabilidade
-----------
Result
Nenhum log de saída disponível para este tool_call.
A probabilidade de obter o número 5 exatamente 400 vezes ao lançar um dado de 8 lados 1000 vezes é incrivelmente baixa, aproximadamente \(4.62 \times 10^{-106}\). Isso é um número muito próximo de zero, o que indica que é um evento extremamente improvável.

=== Step: message_creation


C:\Users\chris\AppData\Local\Temp\ipykernel_10844\3242390766.py:16: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.retrieve(


A probabilidade de obter o número 5 exatamente 400 vezes ao lançar um dado de 8 lados 1000 vezes é incrivelmente baixa, aproximadamente \(4.62 \times 10^{-106}\). Isso é um número muito próximo de zero, o que indica que é um evento extremamente improvável.
